# 马尔可夫决策过程
## 马尔可夫过程
### 随机过程
**随机过程**(stochastic process) 是概率论的“动力学”部分。
- 概率论：概率论的研究对象是静态的随机现象
- 随机过程：随机过程的研究对象是随机现象的动态演变规律（例如天气随时间的变化、城市交通随时间的变化等）
在随机过程中，随机现象在某时刻$t$的取值是一个向量随机变量，用$S_t$表示，所有可能的状态组成状态空间$S$。随机现象辨识状态的变化过程。在某时刻$t$的状态$S_t$通常取决于$t$时刻之前的状态。我们将已知历史信息$(S_1,\ldots,S_{t-1})$时下一个时刻状态为$S_{t+1}$的概率表示成 $P(S_{t+1} | S_1,\ldots,S_t)$。
### 马尔可夫性质
当且仅当某时刻的状态只取决于上一时刻的状态时，一个随机过程可称为具有马尔可夫性质(Markov property)，用公式表示为$P(S_{t+1} | S_{t})=P(S_{t+1}|S_1,\ldots,S_{t})$ 。
也就是说，当前状态是未来的充分统计量，即下一个状态只取决于当前状态，而不会收到过去状态的影响。
> 需要明确的是，具有马尔可夫性并不代表这个随机过程就和历史完全没有关系。因为虽然$t+1$时刻的状态只与$t$时刻的状态有关，但是$t$时刻的状态其实包含了$t-1$时刻的状态信息，通过这种链式的关系，历史的信息被传递到现在。
### 马尔可夫过程
**马尔可夫过程**(Markov process) 指具有马尔可夫性质的随机过程，也被称为**马尔可夫链**(Markove chain)。我们通常用元组$(S,P)$来表示一个马尔可夫过程，其中$S$是状态空间，$P$是状态转移概率矩阵。状态转移概率矩阵$P$是一个矩阵，其中$P_{ij}$表示从状态$i$转移到状态$j$的概率。

In [2]:
import numpy as np
np.random.seed(0)
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0]
gamma = 0.5     # 折扣因子


def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = gamma * G + rewards[chain[i]-1]
    return G


chain =[1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print("Return starting from state 1: ", G)

Return starting from state 1:  -2.5


### 价值函数
**价值** (Value)：在马尔可夫奖励过程中，一个状态的期望回报（即从这个状态触发的未来累积奖励的期望）被称为这个状态的价值
所有状态的价值就组成了**价值函数** (Value Function)。价值函数的输入为某个状态，输出为这个状态的价值。价值函数写成：$$V(s)=\mathbb{E}[G_{t}|S_{t}=s]$$

In [3]:
def compute(P, rewards, gamma, states_num):
    rewards = np.array(rewards).reshape((-1,1)) # 将rewards写成列向量形式
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P), rewards)
    return value

In [4]:
V = compute(P, rewards, gamma, 6)
print("MRP中每个状态价值分别为\n",V) 

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


## 马尔可夫决策过程
马尔可夫过程和马尔可夫奖励过程都是**自发改变**的随机过程；而如果有一个外界的“刺激”来共同改变这个随机过程，就有了**马尔可夫决策过程**(Markov decision process, MDP)。
MDP和MRP非常相像，主要区别为MDP中的状态转移函数和奖励函数都比MRP多了动作$a$作为自变量。
> **注意**：
> 在上面MDP的定义中，不在使用类似MRP定义中的状态转移矩阵方式，而是直接表示成状态转移函数。
> 这样做意识因为此时状态转移与动作也相关，变成了一个三维数组，而不是一个二维矩阵；
> 二是因为状态转移函数更具有一般意义，例如，如果状态集合不是有限的，就无法用数组表示，但仍然可以用状态转移函数表示

### 策略
智能体的策略(Policy)通常用字母$\pi$表示。策略$\pi(a|s)=P(A_t=a|S_t=s)$ 是一个函数，表示在输入状态$s$情况下采取动作$a$的概率。
1. **确定性策略**(deterministic policy)
在每个状态时只输出一个确定性的动作，即只有该动作的概率为1，其他动作概率为0；
2. **随机性策略**(stochastic policy)
3. 在每个状态时输出的是关于动作的概率分布，然后根据该分布进行采样就得到一个动作

### 状态价值函数
$V^{\pi}(s)$ 表示在MDP中基于策略$\pi$的状态极值函数(state-value funtion)，定义为从状态$s$触发遵循策略$\pi$能获得的期望回报$$V^\pi(s)=\mathbb{E}_\pi[G_t|S_t=s]$$

### 动作价值函数
动作价值函数(action-value function)，用$Q^{\pi}(s,a)$表示在MDP遵循策略$\pi$时，对当前状态$s$执行动作$a$得到的期望回报$$Q^\pi(s,a)=\mathbb{E}_\pi[G_t|S_t=s,A_t=a]$$
状态价值函数和动作价值函数之间的关系：
$$V^{\pi}(s)=\sum_{a\in A}\pi(a|s)Q^{\pi}(s,a)$$
使用策略$\pi$时，状态$s$下采取动作$a$的价值等于即时奖励加上经过衰减后的所有可能的下一个状态的状态转移概率与相应的价值的乘积：
$$Q^\pi(s,a)=r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^\pi(s^{\prime})$$
> **注意**
> 此处的状态转移概率依旧存在的意义：即使采取确定的策略，但是下一个状态依旧不确定，但也可能是确定的，此时转移概率为1。

### 贝尔曼期望方程
